In [1]:
import keras
import tensorflow

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np


In [3]:
x,y= mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [5]:
img_rows, img_cols = 28, 28
channels=1
img_shape = (img_rows, img_cols, channels)


In [6]:
def build_generator():
  noise_shape=(100,)  # a one dimensional vector of noise of size 100
  model=Sequential()

  model.add(Dense(256,input_shape=noise_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))


  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))


  #multiply the image dimensions to obtain the total number of pixels
  #tanh reduces intensity to between -1 and 1
  model.add(Dense(np.prod(img_shape),activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()


  #input layer shoul dhave noise shape
  noise=Input(shape=noise_shape)


  #the generated image is returned by the model that takes the noise as input
  img=model(noise)

  return Model(noise,img)





In [7]:
def build_discriminator():
  model=Sequential()

  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))


  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))


  model.add(Dense(1,activation='sigmoid'))

  model.summary()

  img=Input(shape=img_shape)

  validity=model(img)

  return Model(img,validity)  #since sigmoid returns a value between 0 and 1, gives an indication

In [8]:
#save_interval= number of intervals at which to save the generated images


def train(epochs,batch_size,save_interval=500):


  #load the real mnist dataset

  (X_train,_), (_,_) = mnist.load_data()

  #rescale

  X_train=(X_train.astype(np.float32)-127.5)/127.5

  #add channel dimension
  X_train=np.expand_dims(X_train,axis=3)

  half_batch=int(batch_size/2)

  for epoch in range(epochs):


    #Train the discriminator first and then the generator. go back and forth


    #chooose a random half of images to feed into the discriminator
    idx=np.random.randint(0,X_train.shape[0],half_batch)

    #the real images
    imgs=X_train[idx]


    # generate some random gaussian nose- half_batch number of vectors, each of size 100
    noise=np.random.normal(0,1,(half_batch,100))
    gen_imgs=generator.predict(noise)


     #train the discriminator on the real images
     #The value is set to one for the real images- hence np.ones
    d_loss_real=discriminator.train_on_batch(imgs,np.ones((half_batch,1)))


      #The value is set to zero for the fake images- hence np.zeroes
    d_loss_fake=discriminator.train_on_batch(gen_imgs,np.zeros((half_batch,1)))



      #you'll get two loss values, one for real and one for fake, and
    d_loss=0.5*np.add(d_loss_real,d_loss_fake)





    #Train the generator

    #generate noise again
    noise=np.random.normal(0,1,(batch_size,100))

    #generate truth values
    valid_y=np.ones((batch_size,1))

    #train the generator


    #note that though we are generating noise and the images fed into the generator aren't real, the corresponding label associated with it is taken to be 1 (real).
    #This is how the discriminator is fooled into thinking that the images from the generator are real images.
    g_loss=combined.train_on_batch(noise,valid_y)


    print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))


    if epoch % save_interval == 0:
      save_imgs(epoch)






In [21]:
import os
def save_imgs(epoch):
  save_dir = '/content/images/'
  os.makedirs(save_dir, exist_ok=True)

  #a 5x5 grid of generated images
  r,c=5,5
  noise=np.random.normal(0,1,(r*c,100))
  gen_imgs=generator.predict(noise)


  #scale images to 0-1
  gen_imgs=(0.5+gen_imgs)*127.5

  fig,ax=plt.subplots(r,c)
  cnt=0
  for i in range(r):
    for j in range(c):
      ax[i,j].imshow(gen_imgs[cnt,:,:,0],cmap='gray')
      ax[i,j].axis('off')
      cnt+=1

      fig.savefig('/content/images/%d.png' % epoch)
  plt.close()

In [23]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam


discriminator = build_discriminator()
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


generator = build_generator()


z = tf.keras.layers.Input(shape=(100,))
img = generator(z)


discriminator.trainable = False


valid = discriminator(img)


combined = tf.keras.models.Model(z, valid)


discriminator_vars = discriminator.trainable_variables
generator_vars = generator.trainable_variables


optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
optimizer.build(discriminator_vars + generator_vars)

combined.compile(loss='binary_crossentropy', optimizer=optimizer)


train(epochs=50000, batch_size=32)

# Save the generator model
generator.save('generator.h5')


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 20ms/step
47502 [D loss: 0.698591, acc.: 43.75%] [G loss: 0.852771]
1/1 [==============================] - 0s 20ms/step
47503 [D loss: 0.756979, acc.: 46.88%] [G loss: 0.875845]
1/1 [==============================] - 0s 21ms/step
47504 [D loss: 0.660174, acc.: 59.38%] [G loss: 0.805200]
1/1 [==============================] - 0s 20ms/step
47505 [D loss: 0.681057, acc.: 65.62%] [G loss: 0.793453]
1/1 [==============================] - 0s 20ms/step
47506 [D loss: 0.655487, acc.: 62.50%] [G loss: 0.843973]
1/1 [==============================] - 0s 20ms/step
47507 [D loss: 0.691238, acc.: 53.12%] [G loss: 0.825072]
1/1 [==============================] - 0s 20ms/step
47508 [D loss: 0.727293, acc.: 56.25%] [G loss: 0.905390]
1/1 [==============================] - 0s 20ms/step
47509 [D loss: 0.733828, acc.: 40.62%] [G loss: 0.906155]
1/1 [==============================] - 0s 20ms/step
47510 [D loss: 0.

49999 [D loss: 0.700592, acc.: 62.50%] [G loss: 0.779529]


In [24]:
!zip -r /content/file.zip /content/images

  adding: content/images/ (stored 0%)
  adding: content/images/9500.png (deflated 8%)
  adding: content/images/29000.png (deflated 9%)
  adding: content/images/13500.png (deflated 8%)
  adding: content/images/37000.png (deflated 8%)
  adding: content/images/16500.png (deflated 9%)
  adding: content/images/27500.png (deflated 9%)
  adding: content/images/29500.png (deflated 10%)
  adding: content/images/10500.png (deflated 9%)
  adding: content/images/4000.png (deflated 6%)
  adding: content/images/48500.png (deflated 9%)
  adding: content/images/21500.png (deflated 9%)
  adding: content/images/30000.png (deflated 9%)
  adding: content/images/32500.png (deflated 9%)
  adding: content/images/500.png (deflated 7%)
  adding: content/images/27000.png (deflated 9%)
  adding: content/images/43500.png (deflated 8%)
  adding: content/images/26000.png (deflated 9%)
  adding: content/images/41500.png (deflated 9%)
  adding: content/images/38500.png (deflated 10%)
  adding: content/images/39500.pn

In [25]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>